### 1. 사용환경 준비 (OpenAI)
- API-key는 .env에서 불러옴

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPEN_API_KEY")

### 2. 모델 로드하기
- 사용한 모델 : gpt-4o-mini
- 명시적으로 불러온 api-key를 할당
- 논문에 대한 질문의 답변을 해주는 모델이므로 temperature을 0으로 설정해 창의성을 억제함

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(temperature=0, model="gpt-4o-mini", api_key=api_key)

### 3. 문서 로드하기 (초거대 언어모델 연구 동향.pdf)

In [1]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/초거대 언어모델 연구 동향.pdf")

# 페이지 별 문서 로드
docs = loader.load()

#### 다루지 못하는 metaData제거
- 표 내용을 PyPDFLoader를 이용한 텍스트 변환을 하게 되면 제대로 열에 따른 띄어쓰기를 하지 못함을 발견
- 때문에 Hallucination을 일으킬 수 있는 표 데이터 삭제를 전처리 단계에서 진행함

In [161]:
import re

def remove_table_section(text):
    # "표"라는 단어가 포함된 부분부터 그 이후의 내용 제거
    text_without_table = re.split(r"\n표", text, maxsplit=1)[0]
    return text_without_table.strip()

for doc in docs:
    doc.page_content = remove_table_section(doc.page_content)

docs

[Document(metadata={'source': '초거대 언어모델 연구 동향.pdf', 'page': 0}, page_content='8 특집원고  초거대 언어모델 연구 동향\n초거대 언어모델 연구 동향\n업스테이지  박찬준*･이원성･김윤기･김지후･이활석\n \n1. 서  론1)\nChatGPT1)와 같은 초거대 언어모델(Large Language \nModel, LLM) 의 등장으로 기존에 병렬적으로 연구되\n던 다양한 자연언어처리 하위 분야들이 하나의 모델\n로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 \n발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의\n응답, 형태소분석 등의 작업을 모두 처리할 수 있게 \n되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하\n느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이\n에 따라 사용자의 목적에 맞는 출력을 생성하는 패러\n다임을 맞이하게 되었다 [1].\nLLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 \n발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 \n요인에 기반하고 있으며, 이 요인들은 현대 자연언어\n처리 (Natural Language Processing, NLP) 연구의 핵심\n적인 추세로 간주된다. 첫째로, 데이터의 양적 확대는 \n무시할 수 없는 중요한 요인이다. 디지털화의 선도로, \n텍스트 데이터의 양이 기하급수적으로 증가하였고, \n이는 연구의 질적 변화를 가져왔다. 대규모 코퍼스의 \n활용은 LLM의 일반화 능력을 향상시키며, 다양한 맥\n락과 주제에 대한 깊은 학습을 가능하게 한다. 둘째\n로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정\n적이었다. 특히, Graphics Processing Unit (GPU) 및 \nTensor Processing Unit (TPU) 와 같은 고성능 병렬 처\n리 하드웨어의 개발은 모델 학습에 있어 병목 현상을 \n크게 완화시켰다. 이로 인해 연구자들은 모델의 복잡\n성을 키우고, 더욱 깊은 신경망 

### 4. 문서 청크로 나누기
- chunk_size=100: 각 청크의 길이를 100글자로 지정함 이때 한국어, 영어 모두 같은 동일하게 100글자 제한이 적용됨, 논문의 문장의 길이를 살펴본 결과 짧은 문장 구분을 하고 있다는 특징을 파악해 이에적합한 청크 길이를 선택함 (문장의 단위 자체 가 긴 글의 경우 보통 512 토큰을 할당한다고 함)
- chunk_overlap=10 : 청크 간 10글자씩 겹침, 앞 뒤로 해당 겹침은 적용이 되며 이는 모델의 청크 간의 문맥을 잃지 않도록 도와줌 (긴 글의 경우 보통 100토큰)
- length_function=len : 각 청크의 길이를 측정할 때 len함수를 이용해 계산함
- is_sperate_regex=False : 구분자로 정규식을 사용하지 않음, 복잡하지 않은 패턴 매칭일 경우에 사용함

---
CharacterTextPlitter : 텍스트를 나눌 때 사용할 구분자를 지정해서 나누는 방법
RecursiveCharacterTextSplitter : 단일 구분자 기준으로 텍스트를 분할하는 것이 아닌 우선순위에 따라 재귀적으로 적용하여 텍스트를 나눔

---

#### 결론 : 불러온 문서는 논문임. 때문에 문장단위 유사도보다 문단별 유사도를 통해 찾는 것이 더 적합하다는 판단을 함

In [65]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=512,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

splits_char = text_splitter.split_documents(docs)
splits_char[:10]

[Document(metadata={'source': '초거대 언어모델 연구 동향.pdf', 'page': 0}, page_content='8 특집원고  초거대 언어모델 연구 동향\n초거대 언어모델 연구 동향\n업스테이지  박찬준*･이원성･김윤기･김지후･이활석\n \n1. 서  론1)\nChatGPT1)와 같은 초거대 언어모델(Large Language \nModel, LLM) 의 등장으로 기존에 병렬적으로 연구되\n던 다양한 자연언어처리 하위 분야들이 하나의 모델\n로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 \n발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의\n응답, 형태소분석 등의 작업을 모두 처리할 수 있게 \n되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하\n느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이\n에 따라 사용자의 목적에 맞는 출력을 생성하는 패러\n다임을 맞이하게 되었다 [1].\nLLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 \n발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 \n요인에 기반하고 있으며, 이 요인들은 현대 자연언어\n처리 (Natural Language Processing, NLP) 연구의 핵심\n적인 추세로 간주된다. 첫째로, 데이터의 양적 확대는 \n무시할 수 없는 중요한 요인이다. 디지털화의 선도로, \n텍스트 데이터의 양이 기하급수적으로 증가하였고, \n이는 연구의 질적 변화를 가져왔다. 대규모 코퍼스의 \n활용은 LLM의 일반화 능력을 향상시키며, 다양한 맥\n락과 주제에 대한 깊은 학습을 가능하게 한다. 둘째\n로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정\n적이었다. 특히, Graphics Processing Unit (GPU) 및 \nTensor Processing Unit (TPU) 와 같은 고성능 병렬 처\n리 하드웨어의 개발은 모델 학습에 있어 병목 현상을 \n크게 완화시켰다. 이로 인해 연구자들은 모델의 복잡\n성을 키우고, 더욱 깊은 신경망 

In [168]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

# 문서 분할
splits_recur = recursive_text_splitter.split_documents(docs)

In [169]:
splits = splits_recur

### 5. 벡터 임베딩 생성
- 텍스트를 벡터로 변환해줌

### 6. 벡터 스토어 생성
- FAISS : 대규모 벡터 데이터를 효율적으로 저장하고 유사도 검색을 수행함
- 벡터 스토어란 벡터를 저장하고 저장한 벡터를 유사도 기반으로 검색하기 위해 설계된 DB와 비슷한 개념
- 즉, 벡터 스토어가 큰 개념이고, 이를 활용할 수 있는 라이브러리로 FAISS가 존재하는데 특히나 대규모 벡터 데이터의 검색을 위해 최적회된 라이브러리임

In [170]:
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.vectorstores import FAISS

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

### 7. FAISS를 Retriever로 변환
- retriever 사용 이유 : 벡터 유사도 검색을 한 후 (FAISS) 검색 결과를 텍스트 형태로 반환해줘야 하기 때문
- search_type="similarity" : 유사도 기반 검색
- search_kwargs={"k": 5} : 검색에 해당하는 Document를 5개 가져옴 (엄밀히 말하자면 문장의 자세한 정보가 포함되어 있는 5개의 Document)

In [39]:
!pip install rank_bm25

#### RAG 성능 올리기 - Retreiever 활용

- 기본 유사도 기반 FAISS retriever를 사용한 것에 더해 BM25 retriever를 함께 앙상블 시킴
- BM25Retriever는 단어 빈도 기반 점수 계산 retriever임
- 앙상블의 비율은 0.5씩 할당해 단어 빈도 점수, FAISS 기반 유사도가 높은 상위 5개의 문서가 추출됨

In [175]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(docs)
faiss_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, faiss_retriever],
            weights=[0.5, 0.5]  # 가중치 설정 (가중치의 합은 1.0)
        )

### 8. 프롬프트 템플릿 정의
- system : 답변해줄 AI의 역할 및 요구사항 정의
- user : 사용자에게 입력받을 사항 정의


In [172]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

### 9. RAG 체인 구성
1. contextual_prompt로 들어온 question은 rag_chain_debug에서 retriever로 전달됨
2. 관련된 문서 리스트는 input["context"] 형태로 전달됨
3. RunnablePassthrough 클래스는 데이터를 전달하는 역할을 함. invoke() 메서드를 통해 입력된 데이터를 반환함
4. ContextToText 클래스 내의 invoke 함수를 통해 inputs["context"]로 들어온 관련 문서 (Document) 는 문자열로 결합되게 되어 model로는 content와 question을 결합한 딕셔너리 형태로 model에 전달됨
5. DebugPassThrough 클래스 내의 invoke 함수는 디버깅 (어느 페이지에서 어떤 줄의 내용을 근거했는지를 추적하기 위한 용도). ContextToText 클래스와는 다르게 어떠한 가공도 하지 않음
6. 정리하자면 앞서 FAISS를 통해 질문과 관련있는 Document를 가져오고 model은 들어온 질문 데이터 분석과 가져온 Document를 기반으로 한 답변 데이터 생성을 하게 됨

In [176]:
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
    
    
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

### 10. 챗봇 구동 확인

#### 질문
---

아래 질문들의 경우 표 데이터를 제거한 후 제대로 추출할 수 있었던 질문, 표 데이터도 같은 문맥의 문서로 취급되었기 때문



1. Open  Ko-LLM  Leaderboard에는 어떤 기업들이 참여하고 있어?
2. Open  Ko-LLM  Leaderboard에는 카카오가 참여하고 있어?

---

아래 질문들의 경우 retriever에 BM25를 적용함으로 제대로 추출할 수 있었던 질문, 적용하지 않을 시 Hullcination을 일으킴



3. 한국의 LLM 리더보드에 ETRI가 참여하고 있어?
4. 한국의 LLM 리더보드에 카카오가 참여하고 있어?
5. Open  Ko-LLM  Leaderboard에는 카카오가 참여하고 있어?
6. Open  Ko-LLM  Leaderboard에는 ETRI가 참여하고 있어?
7. 카카오의 인공지능 윤리 원칙에 책임성이 포함되어 있어?

In [177]:
while True:
    query = input("질문을 입력하세요! 종료를 원한다면 exit을 입력하세요.")
    if query == "exit":
        break
    
    response = rag_chain_debug.invoke(query)
    print("RAG response : " + response.content)

Debug Output: Open  Ko-LLM  Leaderboard에는 어떤 기업들이 참여하고 있어?
Debug Output: {'context': [Document(metadata={'source': '초거대 언어모델 연구 동향.pdf', 'page': 9}, page_content='2023. 11 정보과학회지 17\n의 경우도 많은 모델들이 OpenLLM Leaderboard에 참\n가하고 있으며, 특히 업스테이지가 두드러진 성과를 \n보였다. 업스테이지는 해당 리더보드에서 두 번이나 \n세계 1위의 자리를 차지한 뛰어난 성과를 보였다. 이\n로 인해 다양한 국내 기업들이 이 리더보드에서의 경\n쟁에 참여하게 되었으며, 국내 LLM 연구 분야 활성\n화에 일조하였다.\n5.2 Open Ko-LLM Leaderboard\n한국어에서도 Open LLM 리더보드가 운영되고 있다. \nO p e n  K o - L L M  L e a d e r b o a r d21)라는 이름으로 NIA와 \n업스테이지에서 공동 주관을 하고 있으며, KT Cloud\n의 인프라 지원으로 운영되고 있다 . Ko-HellaSwag, \nKo-MMLU, Ko-Arc, Ko-Truthful QA, Ko-CommonGen \nV2의 총 5가지 태스크로 운영되고 있다. 기존 영어 \nOpenLLM Leaderboard에서 운영하고 있는 4개의 태\n스크를 한국어화 시킨 데이터에, 고려대학교 자연언\n어처리 연구실에서 구축한 Ko-CommonGen V2 밴치\n마크 데이터셋을 추가하여, 평가 지표로 활용하고 있\n는 리더보드이다.\n해당 리더보드는 오픈 후 2주만에 100개가 넘는 모\n델들이 참여할 뿐만 아니라, 한국의 대표적인 Open \nLLM인 Polyglot-Ko [38], KULLM 22), KoAlpaca 23)와 \n더불어 42MARU24), ETRI 25), Maum.AI 26) 등 다양한 \n21) https://huggingface.co/spaces/upstage/op

### Non-RAG 모델과의 비교
- Hallucination을 일으킴

In [178]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# 모델 초기화
model_raw = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", api_key=api_key)

In [180]:
# 시스템 메시지 설정
system_message = SystemMessage(content="너는 최근 초거대 언어모델 연구 동향을 알려주는 인공지능이야. 질문에 알맞은 답변을 해줘.")
messages = [system_message]

while True:
    # 유저 입력
    user_input = input("질문을 입력하세요 : ")
    if user_input.lower() == "exit":
        break
    print("question: " + user_input)
    # 사용자 메시지 추가
    messages.append(HumanMessage(content=user_input))
    
    # ChatOpenAI 모델을 이용해 답변 생성
    response = model_raw.invoke(messages)
    
    # AI의 답변을 가져와 저장
    reply = response.content
    messages.append(AIMessage(content=reply))
    
    print("Raw AI Response: " + reply)

question: Open  Ko-LLM  Leaderboard에는 어떤 기업들이 참여하고 있어?
Raw AI Response: Open Ko-LLM Leaderboard는 Kakao Brain, Kakao Enterprise, NAVER, AILAB, SK T-Brain, LG CNS, LG Electronics, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG Research, SAMSUNG SDS, SAMSUNG Electronics, SAMSUNG 

#### 왜 RAG 이 필요한가

- RAG를 통해 LLM 모델의 Hallucination을 줄일 수 있으며 업데이트 되지 않은 최신 소식 / 답변하고자 하는 최신 소식 / 본인 서비스에 대한 사적인 정보에 대한 답변을 할 수 있기 때문임